# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["lat", "lon"]].astype(float)
humidity = city_data["humidity"]

In [ ]:
# Create the heat map
# Set the layout to see the majority of the data points without duplication
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=(0.00, 0.00), zoom_level = 2, layout=figure_layout)

# add the heat layer showing cities with their humidity as weight.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)
fig.add_layer(heat_layer)

# show the map
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal weather conditions are:
* Between 40 and 50 degrees
* Medium humidity - no more than 50%
* Light breeze only (5 mph or less)


In [ ]:
ideal_city_data = city_data.loc[((city_data["maxtemp"] >= 40) 
                                & (city_data["maxtemp"] <=50) 
                                & (city_data["humidity"] <= 50)
                                & (city_data["windspeed"] <= 5))]
ideal_city_data

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [2]:
hotel_df = ideal_city_data
hotel_df["Hotel Name"] = ""
hotel_df.head()

target_radius = 5000
target_type = "lodging"

for index, row in ideal_city_data:
    target_coordinates = (row["lat"], row["lon"])
    print(city)
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    hotel_df["Hotel Name"] = response["name"]
hotel_df

NameError: name 'ideal_city_data' is not defined

In [ ]:
# Create a map showing hotels on the previous fig
marker_locations = [['Latitude', 'Longitude']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
